# Experimental measurement of a Bell inequality <br> (CHSH combination of correlators)
Introduction: The purpose of this computational exercise is to measure the CHSH correlators that test whether nature is (or is not) compatible with local realism by respecting (violating, as quantum mechanics predicts) the CHSH inequality. We will first simulate the experiment using qiskit's aer simulator and then conduct the actual quantum experiment using the qubits of a quantum computer.

### PREAMBLE  

In [1]:
!pip install qiskit qiskit_aer qiskit_ibm_runtime matplotlib pylatexenc --quiet

In [3]:
# Importing standard Qiskit libraries
from qiskit import QuantumCircuit, transpile
from qiskit_aer import Aer # Aer simulator
from qiskit.visualization import *
from qiskit_ibm_runtime import QiskitRuntimeService, Sampler, Estimator, Session, Options
from qiskit.quantum_info import Statevector, Pauli, SparsePauliOp
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

import numpy as np
from numpy import sqrt as sqrt
np.set_printoptions(precision=7)

In [5]:
# Loading the aer simulator
backend_S = Aer.get_backend("aer_simulator")

### The CHSH inequality:
If $\{A_1, A_2\}$ and $\{B_1,B_2\}$ are two pairs of observables (with dichotomic/binary outcome) of two spatially separated systems, the expected values of their products $\langle A_i B_j \rangle$ according to any local hidden variable model (that is, an attempt at trying to explain away quantum features with additional classical mechanics variables) satisfy the classical CHSH inequality

$$
|\langle A_1 B_1\rangle +\langle A_1 B_2\rangle + \langle A_2 B_1\rangle - \langle A_2 B_2\rangle|\leq 2 .
$$

Quantum theory, on the contrary, predicts that this inequality is violated for a suitable choice of observables, obtaining the maximum violation, in the case of two qubits, when 

$$
A_1=X,\quad A_2=Y, \quad B_1=\frac{-(X+Y)}{\sqrt{2}}, \quad B_2=\frac{-(X-Y)}{\sqrt{2}} 
$$
or rotationally equivalent configurations. Here $X$ and $Y$ denote the $\sigma_x$ and $\sigma_y$ Pauli matrices.

The command "SparsePauliOp.from_list([('O1', c1), ('O2', c2),...])" takes as input the coefficients (simply numbers) "c1, c2, ..." and one or more Pauli matrices written as a text string  "O1, O2,..." (e.g. O1=XYYXZ$\equiv\sigma_x\otimes \sigma_y \otimes \sigma_y\otimes \sigma_x \otimes \sigma_z$). On output it produces the operator c1 O1 + c2 O2 + ... 
 
#### <font color=teal> **Question 2**: Using this command, define the four product observables that appear in the CHSH inequality: $A_1B_1$, $A_2B_1$, $A_1B_2$, and $A_2B_2$: </font>

In [9]:
# Define pairs of observables for maximum violation of the CHSH innequality
A1B1=SparsePauliOp.from_list([('XX', -1/sqrt(2)), ('XY', -1/sqrt(2))]) 
A1B2=SparsePauliOp.from_list([('XX', -1/sqrt(2)), ('XY', 1/sqrt(2))])
A2B1=SparsePauliOp.from_list([('YX', -1/sqrt(2)), ('YY', -1/sqrt(2))])
A2B2=SparsePauliOp.from_list([('YX', -1/sqrt(2)), ('YY', 1/sqrt(2))])
Obs=[A1B1,A1B2,A2B1,A2B2] #The four correlators are packaged in one list of observables

### Initialization of the circuit
#### <font color=teal> **Question 1**:  Define a circuit that prepares the "singlet state" of two qubits $|\Psi^-\rangle=\tfrac{1}{\sqrt2}(|01\rangle-|10\rangle)$.

In [12]:
# Prepare the input circuit:
chsh_circuit = QuantumCircuit(2)
chsh_circuit.h(0)
chsh_circuit.cx(0,1)
chsh_circuit.z(0)
chsh_circuit.x(0)
chsh_circuit.draw(output='mpl')
phi=Statevector(chsh_circuit)
phi.draw(output='latex')

<IPython.core.display.Latex object>

### Run the quantum circuit on the simulation machine

The idea is to "measure" the observable <font color="blue"> Obs </font> over the state just prepared and stored in <font color="blue">chsh_circuit</font>.

In [14]:
# Choose the "backend" (computational engine doing the work behind)  
estimator_S = Estimator(mode=backend_S) #for the determination of mean values

# Execute the circuit 
job_S = estimator_S.run([(chsh_circuit,Obs)])
result_S = job_S.result()[0]

In [16]:
#Print simulated expectation values
print(f"Simulated expectation values for the four correlators:\n  {result_S.data.evs}")

Simulated expectation values for the four correlators:
  [ 0.707452   0.7067615  0.6991656 -0.7150479]


With those four number we can check whether the CHSH inequality is or is not violated;<br> however, we would like to know the uncertainty in the test, so we need to get some more data from the various shots (repetitions of the quantum measurement on the computer).

In [19]:
# Retrieve sample variance and number of shots from metadata
StandardDeviations = result_S.data.stds
std_S1=round(StandardDeviations[0],3)
std_S2=round(StandardDeviations[1],3)
std_S3=round(StandardDeviations[2],3)
std_S4=round(StandardDeviations[3],9) #note the different rounding off here, for checking


# Calculate and print simulated standard errors
Standard_errors_S = [float(std_S1),float(std_S2),float(std_S3),float(std_S4)]
print(f"Simulated standard errors: {Standard_errors_S}")

Simulated standard errors: [0.011, 0.011, 0.011, 0.011047847]


#### <font color=teal> **Question 3**: Calculate the simulated value obtained for the CHSH inequality jointly with its error </font>

In [23]:
CHSH_mean=result_S.data.evs[0]+result_S.data.evs[1]+result_S.data.evs[2]-result_S.data.evs[3]
CHSH_uncertainty=sqrt(Standard_errors_S[0]**2+Standard_errors_S[1]**2+Standard_errors_S[2]**2+Standard_errors_S[3]**2)
print(f"The simulated result is {CHSH_mean} + {CHSH_uncertainty}:\ndoes it exceed 2 with sufficient statistical certainty?")

The simulated result is 2.82842712474619 + 0.022023962480339658:
does it exceed 2 with sufficient statistical certainty?


### Run the quantum circuit on a real quantum machine

IBM offers free computation time accesible throught the cloud, each account is able to generate "user tokens" that allow to send "jobs" to real quantum hardware. Please copy your token number here (to generate it see instructions in the pdf)

In [25]:
My_user_token = "b9968ad2d40bcf52dc390f83272649228d8fbb847bc6d72190e27e1f4bbeabf1c8f422151cc4ea587452529d7cbbb1ac3dfd527b2f917b18c18e073accd7bdb3" # Use this at home (long waiting time

##### Defining the quantum backend

In [28]:
# Quantum machine
Quantum_service = QiskitRuntimeService(channel="ibm_quantum",token=My_user_token)
Quantum_service.usage()

{'period': {'start': '2025-04-01T00:00:00.000Z',
  'end': '2025-04-30T23:59:59.999Z'},
 'byInstance': [{'instance': 'ibm-q/open/main',
   'quota': 600,
   'usage': 3,
   'pendingJobs': 0,
   'maxPendingJobs': 3}]}

First we have to choose one of the available machines: we let the system do it with some requirements

In [31]:
backend_Q = Quantum_service.least_busy(operational=True, simulator=False, min_num_qubits=20)

In [33]:
backend_Q.status()

##### Sending jobs to the machine <br>
<font color="red"> Executing the following cell will cause a charge on your free monthly time allotment, currently 10 min. courtesy of IBM</font>

In [36]:
# Maximum execution time in seconds
my_options = {"max_execution_time": 20}
# Number of shots
pm = generate_preset_pass_manager(backend=backend_Q, optimization_level=1) #Define pass manager
CHSH_circuits = pm.run(chsh_circuit) #copy the circuit 4 times and run against the pass manager
CHSH_observables = [ob.apply_layout(CHSH_circuits.layout) for ob in Obs] #apply transpiled circuit layout to observables
estimator_Q = Estimator(mode=backend_Q,options=my_options) #for the determination of mean values
# Execute the circuit 
job_Q = estimator_Q.run([(CHSH_circuits, CHSH_observables)])

##### Checking job state

In [43]:
# The following cell gives information about the job
job_Q.status() # Status

'DONE'

In [45]:
result_Q = job_Q.result()[0]

In [47]:
job_Q.job_id() # Identification code

'czzryzbkzhn0008dgpd0'

In [49]:
#Once the program has run we can print the simulated expectation values
print(f"Expectation values: {result_Q.data.evs}")

Expectation values: [ 0.7092474  0.7049662  0.7056797 -0.6985444]


In [53]:
# Retrieve sample variance and number of shots from metadata
StandardDeviations = result_Q.data.stds
std_S1=round(StandardDeviations[0],3)
std_S2=round(StandardDeviations[1],3)
std_S3=round(StandardDeviations[2],3)
std_S4=round(StandardDeviations[3],9) #note the different rounding off here, for checking

# Calculate and print standard errors
Standard_errors_Q = [float(std_S1),float(std_S2),float(std_S3),float(std_S4)]
print(f"Standard errors: {Standard_errors_Q}")

Standard errors: [0.013, 0.013, 0.012, 0.012363197]


In [55]:
CHSH_mean_Q=result_Q.data.evs[0]+result_Q.data.evs[1]+result_Q.data.evs[2]-result_Q.data.evs[3]
CHSH_uncertainty_Q=sqrt(Standard_errors_Q[0]**2+Standard_errors_Q[1]**2+Standard_errors_Q[2]**2+Standard_errors_Q[3]**2)
print(f"The simulated result is {CHSH_mean_Q} + {CHSH_uncertainty_Q}:\ndoes it exceed 2 with sufficient statistical certainty?")

The simulated result is 2.8184377252137867 + 0.025196202889737354:
does it exceed 2 with sufficient statistical certainty?


####  <font color=teal> **Question 4**: Compute, with a hand calculator or simple python commands, the experimental value obtained for the CHSH inequality jointly with its error from the quantum data obtained. </font> 

##### Retrieving job and results from  queued and past jobs

If the job was queued for long or we need to check a past one, we can retrieve it if we know its job number, which is displayed in the IBM quantum webpage

In [ ]:
job_number = ""
job =  Quantum_service.job(job_number)
result = job.result()[0].data.evs
result